# SHO Fitter Speed Measurements Benchmarking

In [ ]:
import sys
sys.path.append('../../')

In [ ]:
import numpy as np
import h5py
import time

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from scipy.signal import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from src.m3_learning.optimizers.AdaHessian import AdaHessian
from src.m3_learning.nn.SHO_fitter.SHO import SHO_fit_func_torch
from src.m3_learning.be.processing import convert_amp_phase, transform_params, SHO_fit_to_array
from src.m3_learning.util.preprocessing import global_scaler
from src.m3_learning.nn.benchmarks.inference import computeTime

## Loads data

In [ ]:
# Sets path to file
path = r"./"

# Opens the data file
h5_f = h5py.File(path + "data_file.h5", "r+")

# number of pixels in the image
num_pix = h5_f["Measurement_000"].attrs["num_pix"]

num_pix_1d = int(np.sqrt(num_pix))

# Frequency Vector in Hz
frequency_bin = h5_f["Measurement_000"]["Channel_000"]["Bin_Frequencies"][:]

# extracting spectroscopic values
spectroscopic_values = h5_f['Measurement_000']['Channel_000']['Spectroscopic_Values']

# number of DC voltage steps
voltage_steps = h5_f["Measurement_000"].attrs["num_udvs_steps"]

# Resampled frequency vector
wvec_freq = resample(frequency_bin, 80)

# get raw data (real and imaginary combined)
raw_data = h5_f["Measurement_000"]["Channel_000"]["Raw_Data"]
raw_data_resampled = resample(np.array(h5_f["Measurement_000"]["Channel_000"]["Raw_Data"]).reshape(-1, 165), 80, axis=1)

# conversion of raw data (both resampled and full)
amp, phase = convert_amp_phase(raw_data)
amp_resample, phase_resample = convert_amp_phase(raw_data_resampled)

scaled_data = h5_f["Measurement_000"]["Channel_000"]['complex']['scaled_data'][:]
real_resample = h5_f["Measurement_000"]["Channel_000"]['complex']['real_resample'][:]
imag_resample = h5_f["Measurement_000"]["Channel_000"]['complex']['imag_resample'][:]

# scale the real component of input data
scaler_real = global_scaler()
scaled_data_real = scaler_real.fit_transform(real_resample).reshape(-1, 80)

# scale the imaginary component of input data
scaler_imag = global_scaler()
scaled_data_imag = scaler_imag.fit_transform(imag_resample).reshape(-1, 80)

In [ ]:
# create a list for parameters
fit_results_list = SHO_fit_to_array(h5_f["Measurement_000"]["Channel_000"]["Raw_Data-SHO_Fit_000"]["Fit"])

# flatten parameters list into numpy array
fit_results_list = np.array(fit_results_list).reshape(num_pix, voltage_steps, 5)

# exclude the R2 parameter
params = fit_results_list.reshape(-1, 5)[:, 0:4]

# scale the parameters (now takes only 4 parameters, excluding the R2)
params_scaler = StandardScaler()
scaled_params = params_scaler.fit_transform(fit_results_list.reshape(-1, 5)[:, 0:4])

In [ ]:
data_train, data_test, params_train, params_test = train_test_split(
    scaled_data, scaled_params, test_size=0.7, random_state=42
)

params_test_unscaled = params_scaler.inverse_transform(params_test)

## Benchmarking

In [ ]:
manual_seed = np.arange(1, 11, 1)
optimizers = [torch.optim.Adam, AdaHessian]
optimizers_name = ["ADAM", "ADAHESSIAN"]

In [ ]:
for k, optim in enumerate(optimizers):
  with open(f'{optimizers_name[k]}_SHO_speed.txt', 'w') as file:  
    for seed in manual_seed:
      file.write(f'MANUAL SEED: {seed}\n')
      
      for n in range(6, 11):
        
        class SHO_Model(nn.Module):
          def __init__(self):
              super().__init__()

              # Input block of 1d convolution
              self.hidden_x1 = nn.Sequential(
                  nn.Conv1d(in_channels=2, out_channels=8, kernel_size=7),
                  nn.SELU(),
                  nn.Conv1d(in_channels=8, out_channels=6, kernel_size=7),
                  nn.SELU(),
                  nn.Conv1d(in_channels=6, out_channels=4, kernel_size=5),
                  nn.SELU(),
              )

              # fully connected block
              self.hidden_xfc = nn.Sequential(
                  nn.Linear(256, 20),
                  nn.SELU(),
                  nn.Linear(20, 20),
                  nn.SELU(),
              )

              # 2nd block of 1d-conv layers
              self.hidden_x2 = nn.Sequential(
                  nn.MaxPool1d(kernel_size=2),
                  nn.Conv1d(in_channels=2, out_channels=4, kernel_size=5),
                  nn.SELU(),
                  nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
                  nn.SELU(),
                  nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
                  nn.SELU(),
                  nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
                  nn.SELU(),
                  nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
                  nn.SELU(),
                  nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
                  nn.SELU(),
                  nn.AvgPool1d(kernel_size=2),
                  nn.Conv1d(in_channels=4, out_channels=2, kernel_size=3),
                  nn.SELU(),
                  nn.AvgPool1d(kernel_size=2),
                  nn.Conv1d(in_channels=2, out_channels=2, kernel_size=3),
                  nn.SELU(),
                  nn.AvgPool1d(kernel_size=2),
              )

              # Flatten layer
              self.flatten_layer = nn.Flatten()
              
              # Final embedding block - Output 4 values - linear
              self.hidden_embedding = nn.Sequential(
                  nn.Linear(26, 16),
                  nn.SELU(),
                  nn.Linear(16, 8),
                  nn.SELU(),
                  nn.Linear(8, 4),
              )

          def forward(self, x, n=-1):
            x = torch.swapaxes(x, 1, 2) # output shape - samples, (real, imag), frequency
            x = self.hidden_x1(x)
            xfc = torch.reshape(x, (n, 256)) # batch size, features
            xfc = self.hidden_xfc(xfc)
            x = torch.reshape(x, (n, 2, 128)) # batch size, (real, imag), timesteps
            x = self.hidden_x2(x)
            cnn_flat = self.flatten_layer(x)
            encoded = torch.cat((cnn_flat, xfc), 1) # merge dense and 1d conv.
            embedding = self.hidden_embedding(encoded) # output is 4 parameters

            # corrects the scaling of the parameters
            unscaled_param = embedding*torch.tensor(params_scaler.var_[0:4]**0.5).cuda() \
                                    + torch.tensor(params_scaler.mean_[0:4]).cuda()

            # passes to the pytorch fitting function 
            fits = SHO_fit_func_torch(unscaled_param, wvec_freq, device='cuda')

            # extract and return real and imaginary      
            real = torch.real(fits)
            real_scaled = (real - torch.tensor(scaler_real.mean).cuda())\
                                              /torch.tensor(scaler_real.std).cuda()
            imag = torch.imag(fits)
            imag_scaled = (imag - torch.tensor(scaler_imag.mean).cuda())\
                                              /torch.tensor(scaler_imag.std).cuda()
            out = torch.stack((real_scaled, imag_scaled), 2)
            return out

        ##########################################################################################################
        # TRAINING
        torch.manual_seed(seed)
        np.random.seed(seed)
        torch.cuda.empty_cache()
        model = SHO_Model().cuda().double()

        loss_func = torch.nn.MSELoss()
        batch_size = 2**n
        optimizer = optim(model.parameters())
        train_dataloader = DataLoader(data_train, batch_size=batch_size)
        epochs = 5
        
        start_time_training = time.time()
        for epoch in range(epochs):
          start_time = time.time()
          train_loss = 0.
          total_num = 0

          model.train()

          for train_batch in train_dataloader:
            pred = model(train_batch.double().cuda())
            optimizer.zero_grad()
            loss = loss_func(train_batch.double().cuda(), pred)
            loss.backward(create_graph=True)
            train_loss += loss.item() * pred.shape[0]
            total_num += pred.shape[0]
            optimizer.step()

          train_loss /= total_num
          torch.save(model, f'Trained Models/SHO Fitter/model_{optimizers_name[k]}_{batch_size}.pt')
          torch.save(model.state_dict(), f'Trained Models/SHO Fitter/model_{optimizers_name[k]}_{batch_size}.pth')

          file.write("epoch : {}/{}, recon loss = {:.8f}\n".format(epoch + 1, epochs, train_loss))
          file.write("--- %s seconds ---\n" % (time.time() - start_time))

        file.write(f"Training with batch size={batch_size} took {time.time() - start_time_training} seconds\n\n")

        torch.cuda.empty_cache()
        train_dataloader_valid = DataLoader(scaled_data, batch_size=batch_size)

        # Computes the inference time
        file.write(f"Inference time with batch size={batch_size}\n")
        file.write(computeTime(model, next(iter(train_dataloader_valid)).double(), batch_size=batch_size, write_to_file=True)+'\n')

        ##########################################################################################################
        # prediction of reconstructions
        batch_size = 5000
        train_dataloader = DataLoader(scaled_data, batch_size=batch_size)

        num_elements = len(train_dataloader.dataset)
        num_batches = len(train_dataloader)
        predictions = torch.zeros_like(torch.tensor(scaled_data))

        for i, train_batch in enumerate(train_dataloader):
          start = i*batch_size
          end = start + batch_size

          if i == num_batches - 1:
            end = num_elements

          pred_batch = model(train_batch.double().cuda())
          predictions[start:end] = pred_batch.cpu().detach()

          del pred_batch
          del train_batch
          torch.cuda.empty_cache()

        file.write('Reconstruction error: ' + str((mean_squared_error(scaled_data[:, :, 0], predictions[:, :, 0]) + mean_squared_error(scaled_data[:, :, 1], predictions[:, :, 1]))/ 2.0) + '\n')

        ##########################################################################################################
        # ARCHITECTURE OF PARAMS MODEL
        class SHO_Model(nn.Module):
          def __init__(self):
              super().__init__()

              # Input block of 1d convolution
              self.hidden_x1 = nn.Sequential(
                  nn.Conv1d(in_channels=2, out_channels=8, kernel_size=7),
                  nn.SELU(),
                  nn.Conv1d(in_channels=8, out_channels=6, kernel_size=7),
                  nn.SELU(),
                  nn.Conv1d(in_channels=6, out_channels=4, kernel_size=5),
                  nn.SELU(),
              )

              # fully connected block
              self.hidden_xfc = nn.Sequential(
                  nn.Linear(256, 20),
                  nn.SELU(),
                  nn.Linear(20, 20),
                  nn.SELU(),
              )

              # 2nd block of 1d-conv layers
              self.hidden_x2 = nn.Sequential(
                  nn.MaxPool1d(kernel_size=2),
                  nn.Conv1d(in_channels=2, out_channels=4, kernel_size=5),
                  nn.SELU(),
                  nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
                  nn.SELU(),
                  nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
                  nn.SELU(),
                  nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
                  nn.SELU(),
                  nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
                  nn.SELU(),
                  nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
                  nn.SELU(),
                  nn.AvgPool1d(kernel_size=2),
                  nn.Conv1d(in_channels=4, out_channels=2, kernel_size=3),
                  nn.SELU(),
                  nn.AvgPool1d(kernel_size=2),
                  nn.Conv1d(in_channels=2, out_channels=2, kernel_size=3),
                  nn.SELU(),
                  nn.AvgPool1d(kernel_size=2),
              )

              # Flatten layer
              self.flatten_layer = nn.Flatten()
              
              # Final embedding block - Output 4 values - linear
              self.hidden_embedding = nn.Sequential(
                  nn.Linear(26, 16),
                  nn.SELU(),
                  nn.Linear(16, 8),
                  nn.SELU(),
                  nn.Linear(8, 4),
              )

          def forward(self, x, n=-1):
            x = torch.swapaxes(x, 1, 2) # output shape - samples, (real, imag), frequency
            x = self.hidden_x1(x)
            xfc = torch.reshape(x, (n, 256)) # batch size, features
            xfc = self.hidden_xfc(xfc)
            x = torch.reshape(x, (n, 2, 128)) # batch size, (real, imag), timesteps
            x = self.hidden_x2(x)
            cnn_flat = self.flatten_layer(x)
            encoded = torch.cat((cnn_flat, xfc), 1) # merge dense and 1d conv.
            embedding = self.hidden_embedding(encoded) # output is 4 parameters

            # corrects the scaling of the parameters
            unscaled_param = embedding*torch.tensor(params_scaler.var_[0:4]**0.5).cuda() + torch.tensor(params_scaler.mean_[0:4]).cuda()
            return unscaled_param
        
        ##########################################################################################################
        # LOADING PARAMS MODEL
        batch_size = 2**n
        torch.cuda.empty_cache()
        model_parameters = SHO_Model().cuda()
        # loads prior trained model
        model_parameters = torch.load(f'Trained Models/SHO Fitter/model_{optimizers_name[k]}_{batch_size}.pt')

        ##########################################################################################################
        # prediction of parameters
        batch_size = 5000
        train_dataloader = DataLoader(data_test, batch_size=batch_size)

        num_elements = len(train_dataloader.dataset)
        num_batches = len(train_dataloader)
        test_pred_params = torch.zeros_like(torch.tensor(params_test))

        for i, train_batch in enumerate(train_dataloader):
          start = i*batch_size
          end = start + batch_size

          if i == num_batches - 1:
            end = num_elements

          pred_batch = model_parameters(train_batch.double().cuda())
          test_pred_params[start:end] = pred_batch.cpu().detach()

          del pred_batch
          del train_batch
          torch.cuda.empty_cache()

        test_pred_params = test_pred_params.view(-1, 4)
        test_pred_params = test_pred_params.cpu().detach().numpy()

        # making numpy array copies of parameters
        test_params_copy = np.copy(params_test_unscaled)
        test_pred_params_copy = np.copy(test_pred_params)

        params_transformed, pred_params_transformed = transform_params(test_params_copy, test_pred_params_copy)

        ###########################################################################################################
        # prediction of parameters
        batch_size = 5000
        train_dataloader = DataLoader(scaled_data, batch_size=batch_size)

        num_elements = len(train_dataloader.dataset)
        num_batches = len(train_dataloader)
        all_pred_params = torch.zeros_like(torch.tensor(params))

        for i, train_batch in enumerate(train_dataloader):
          start = i*batch_size
          end = start + batch_size

          if i == num_batches - 1:
            end = num_elements

          pred_batch = model_parameters(train_batch.double().cuda())
          all_pred_params[start:end] = pred_batch.cpu().detach()

          del pred_batch
          del train_batch
          torch.cuda.empty_cache()

        all_pred_params = all_pred_params.cpu().detach().numpy()

        params_copy = np.copy(params)
        all_pred_params_copy = np.copy(all_pred_params)

        all_params_transformed, all_pred_params_transformed = transform_params(params_copy, all_pred_params_copy)

        all_pred_params_scaled = params_scaler.transform(all_pred_params_transformed)
        all_params_scaled = params_scaler.transform(all_params_transformed)

        file.write('\nResults')
        file.write('Total MSE: ' + str(mean_squared_error(all_params_scaled, all_pred_params_scaled)) + '\n')
        file.write('MSE of Amplitude: ' + str(mean_squared_error(all_params_scaled[:, 0], all_pred_params_scaled[:, 0])) + '\n')
        file.write('MSE of Resonance: ' + str(mean_squared_error(all_params_scaled[:, 1], all_pred_params_scaled[:, 1])) + '\n')
        file.write('MSE of Quality-Factor: ' + str(mean_squared_error(all_params_scaled[:, 2], all_pred_params_scaled[:, 2])) + '\n')
        file.write('MSE of Phase: ' + str(mean_squared_error(all_params_scaled[:, 3], all_pred_params_scaled[:, 3])) + '\n')
        file.write('------------------------------\n\n')
        torch.cuda.empty_cache()